## Formatting the climate data
So it has named dimensions if they don't already

In [41]:
# Libraries
%load_ext autoreload
%autoreload

import numpy as np
import iris
import iris.coord_categorisation
import pandas as pd

import sys
sys.path.append('../')
from libs.plot_maps import *


import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading in the data (ignoring precipation because (a) we don't need it and (b) it has additional dimension coordinates (lat_bnds and lon_bnds) which upset iris.

In [42]:
dir_data     = '../data/'
dir_temp     = '../temp/'
dir_outputs  = '../outputs/climate/'
files      = { 'air'               : 'air.mon.mean.nc',
               'precip'            : 'precip.mon.mean.nc',
               'lightning'         : 'lightn_climatology_otd_mlnha.nc',
               'relative_humidity' : 'rhum.mon.mean.nc'}

input_data = {}



for key, f in files.items():
    print(key)
    try:
        dat = iris.load_cube(dir_data + f)
    except:
        print(f)
        dat = iris.load(dir_data + f)[0]
    input_data[key] = dat

air
precip
precip.mon.mean.nc
lightning
relative_humidity


In [43]:
input_data

{'air': <iris 'Cube' of Monthly Mean Air Temperature at sigma level 0.995 / (degC) (time: 865; latitude: 73; longitude: 144)>,
 'precip': <iris 'Cube' of Average Monthly Rate of Precipitation / (mm/day) (time: 493; latitude: 72; longitude: 144)>,
 'lightning': <iris 'Cube' of lightn / (1) (time: 12; latitude: 360; longitude: 720)>,
 'relative_humidity': <iris 'Cube' of Monthly Mean Relative Humidity at sigma level 0.995 / (%) (time: 865; latitude: 73; longitude: 144)>}

Adding a year coordinate to relative humidity and air files and cutting it off before 2001:

In [44]:
post_2001 = {}
for key in input_data.keys():
    if key != 'lightning':
            # Add coord of year
            iris.coord_categorisation.add_year(input_data[key], 'time', name='year')
            
            # Extract from 2001 onwards
            post_2001[key] = input_data[key].extract(iris.Constraint(year=lambda y: y >= 2001))

## Lightning
Regrid lightning to the mask:

In [45]:
air_temp = input_data['air'].collapsed('time', iris.analysis.MEAN)

input_data['lightning'] = input_data['lightning'].regrid(air_temp, iris.analysis.Linear())

In [46]:
lightning = post_2001['air'].copy()
print(input_data['lightning'])
print(lightning)

lightn / (1)                        (time: 12; latitude: 73; longitude: 144)
     Dimension coordinates:
          time                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          comment_1: LIS/OTD 0.5 Degree High Resolution Monthly Climatology (HRMC)
          comment_2:: non-transient
          comment_3:: (i.e, average lightening in period of oberservation for each month)
          comment_4:: Units: Flash count per km2
          history: Wed Apr  2 19:04:33 2008: ncap -O -s lightn[time,lat,lon]=float(lightn*10000.)...
Monthly Mean Air Temperature at sigma level 0.995 / (degC) (time: 229; latitude: 73; longitude: 144)
     Dimension coordinates:
          time                                                  x              -              -
          latitude                                              -             

In [47]:
lightning = post_2001['air'].copy()
light = lightning.aggregated_by(['year'], iris.analysis.MEAN)
j = 0
for k in range(len(light.coord('year').points) - 1):
    lightning.data[j:j+12,:,:] = input_data['lightning'].data
    print('j: ', j)
    j += 12

# In 2019, we've only reached Aug, so I'll add the data in for this single case
print(lightning.shape[0]-j)
lightning.data[j:,:,:] = input_data['lightning'].data[:(lightning.shape[0]-j), :, :]
print('j: ', j)
post_2001['lightning'] = lightning

j:  0
j:  12
j:  24
j:  36
j:  48
j:  60
j:  72
j:  84
j:  96
j:  108
j:  120
j:  132
j:  144
j:  156
j:  168
j:  180
j:  192
j:  204
j:  216
1
j:  228


#### Saving files (all)

In [48]:
for key in post_2001.keys():
    out = key + '2001-2019_test.nc'
    iris.save(post_2001[key], dir_temp + out)

### Adding coord system

In [49]:
files      = { 'air'               : 'air2001-2019_test.nc',
               'precip'            : "precip2001-2019_test.nc",
               'lightning'         : 'lightning2001-2019_test.nc',
               'relative_humidity' : 'relative_humidity2001-2019_test.nc'}

input_data = {}

for key, f in files.items():
    print(key)
    dat = iris.load_cube(dir_temp + f)
    input_data[key] = dat

air
precip
lightning
relative_humidity


In [50]:
lat = input_data['air'].coord('latitude').points
lon = input_data['air'].coord('longitude').points

In [51]:
# Dim coord for latitude (index: 1)
coord_lat = iris.coords.DimCoord(lat, standard_name='latitude',
                                 coord_system=iris.coord_systems.GeogCS(6371229.0),var_name='latitude',
                                 units='degrees')

# Dim coord for longitude (index: 2)
coord_lon = iris.coords.DimCoord(lon, standard_name='longitude',
                                 coord_system=iris.coord_systems.GeogCS(6371229.0),var_name='longitude',
                                 units='degrees')

In [52]:
print(input_data)

{'air': <iris 'Cube' of Monthly Mean Air Temperature at sigma level 0.995 / (degC) (time: 229; latitude: 73; longitude: 144)>, 'precip': <iris 'Cube' of Average Monthly Rate of Precipitation / (mm/day) (time: 229; latitude: 72; longitude: 144)>, 'lightning': <iris 'Cube' of Monthly Mean Air Temperature at sigma level 0.995 / (degC) (time: 229; latitude: 73; longitude: 144)>, 'relative_humidity': <iris 'Cube' of Monthly Mean Relative Humidity at sigma level 0.995 / (%) (time: 229; latitude: 73; longitude: 144)>}


In [53]:
for key in input_data.keys():
    try:
        input_data[key].remove_coord('latitude')
        input_data[key].remove_coord('longitude')
    
        # Adding dimension coordinates to data
        input_data[key].add_dim_coord(coord_lat, 1)
        input_data[key].add_dim_coord(coord_lon, 2)
    except:
        pass

In [54]:
for key in input_data.keys():
    out = key + '2001-2019.nc'
    print(key)
    iris.save(input_data[key], dir_outputs + out)

air
precip
lightning
relative_humidity
